In [ ]:
#Inicializacion de spark session y creacion de nuestra instancia
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sesion3').getOrCreate()

# sql()
Podemos ejecutar consultas SQL como estamos aconstumbrados sobre nuestro dataframe y nos devolvera el resultado en otro
Dataframe

In [2]:
#Para poder hacer esto primero tenemos que crear nuestro Dataframe
mi_dataframe = spark.read.json('nombres.json')

In [3]:
#Y a partir de este Dataframe hay que crear una vista, lo haremos usando createOrReplaceTempView
mi_dataframe.createOrReplaceTempView('personas')
#Como vemos tenemos que asignarle un nombre a la vista por ejemplo personas

In [4]:
#Ahora ya si que podemos  consultar la vista del siguiente modo:
consulta = spark.sql("Select * from personas")

In [5]:
consulta.show()

+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  34|  Juan|
|  Murcia|  23| Maria|
|Alicante|  19| Laura|
|Alicante|  45|Sandra|
|   Elche|null| Pedro|
+--------+----+------+



In [6]:
#Podemos hacer queries complejas, no solo select, podemos usar where por ejemplo, o cualquier cosa que 
#necesitemos
consulta = spark.sql("Select * from personas where Nombre = 'Juan'")

In [7]:
consulta.show()

+------+----+------+
|Ciudad|Edad|Nombre|
+------+----+------+
| Elche|  34|  Juan|
+------+----+------+



In [8]:
#O por ejemplo sacar las personas que son mayores de edad:
consulta = spark.sql("Select Nombre as Mayor_de_edad from personas where Edad>=18")

In [9]:
consulta.show()

+-------------+
|Mayor_de_edad|
+-------------+
|         Juan|
|        Maria|
|        Laura|
|       Sandra|
+-------------+



In [10]:
#O agrupar por ciudades y sacar la media de edad:
consulta = spark.sql("Select Ciudad,sum(Edad)/count(Ciudad) as Edad_media from personas group by Ciudad")

In [11]:
consulta.show()

+--------+----------+
|  Ciudad|Edad_media|
+--------+----------+
|  Murcia|      23.0|
|   Elche|      17.0|
|Alicante|      32.0|
+--------+----------+



In [12]:
#Podemos hacer todo lo que queramos


In [13]:
#Podemos crear una vista global

mi_dataframe.createGlobalTempView('Nombres_global')

In [14]:
spark.sql("Select * from global_temp.Nombres_global").show()

+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  34|  Juan|
|  Murcia|  23| Maria|
|Alicante|  19| Laura|
|Alicante|  45|Sandra|
|   Elche|null| Pedro|
+--------+----+------+



In [15]:
#Creamos una nueva sesion:
spark2 = spark.newSession()

In [16]:
#Vemos que esta vista existe en la sesion nueva:
spark2.sql("Select * from global_temp.Nombres_global").show()

+--------+----+------+
|  Ciudad|Edad|Nombre|
+--------+----+------+
|   Elche|  34|  Juan|
|  Murcia|  23| Maria|
|Alicante|  19| Laura|
|Alicante|  45|Sandra|
|   Elche|null| Pedro|
+--------+----+------+



In [17]:
#Vemos que la vista anterior que era temporal, sigue existiendo en la vista anterior:
spark.sql("Select Ciudad,sum(Edad)/count(Ciudad) as Edad_media from personas group by Ciudad").show()

+--------+----------+
|  Ciudad|Edad_media|
+--------+----------+
|  Murcia|      23.0|
|   Elche|      17.0|
|Alicante|      32.0|
+--------+----------+



In [18]:
#Pero no existe en la nueva
spark2.sql("Select Ciudad,sum(Edad)/count(Ciudad) as Edad_media from personas group by Ciudad").show()


AnalysisException: Table or view not found: personas; line 1 pos 57;
'Aggregate ['Ciudad], ['Ciudad, ('sum('Edad) / 'count('Ciudad)) AS Edad_media#138]
+- 'UnresolvedRelation [personas], [], false
